In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Imports & Setup

In [2]:
import pandas as pd
import numpy as np
from pathlib import Path

np.random.seed(42)

Load USD/LKR Monthly Exchange Rate Data

In [3]:
usd_lkr = pd.read_csv(
    '/content/drive/MyDrive/Coffee/Dev/C2/Data/USD LKR rates/USD_LKR Historical Data.csv'
)

usd_lkr['Date'] = pd.to_datetime(usd_lkr['Date'], format='%m/%d/%Y')
usd_lkr['year'] = usd_lkr['Date'].dt.year
usd_lkr['month'] = usd_lkr['Date'].dt.month

usd_lkr = usd_lkr.rename(columns={'Price': 'usd_lkr_rate'})
usd_lkr = usd_lkr[['year', 'month', 'usd_lkr_rate']]

usd_lkr.head()


,year,month,usd_lkr_rate
0,2025,12,309.875
1,2025,11,308.175
2,2025,10,304.450
3,2025,9,302.550
4,2025,8,302.050


Load Global Coffee Price (Daily, USD/lb)

In [4]:
coffee_global = pd.read_csv(
    '/content/drive/MyDrive/Coffee/Dev/C2/Data/Global coffee index/chart_20251230T014827.csv'
)

coffee_global['Date'] = pd.to_datetime(coffee_global['Date'], format='%m/%d/%Y')
coffee_global['year'] = coffee_global['Date'].dt.year
coffee_global['month'] = coffee_global['Date'].dt.month

coffee_global = coffee_global[
    (coffee_global['year'] >= 2023) &
    (coffee_global['year'] <= 2026)
]


Aggregate Global Coffee Prices

In [5]:
monthly_global_price = (
    coffee_global
    .groupby(['year', 'month'])
    .agg(global_price_usd_lb=('Value', 'mean'))
    .reset_index()
)

monthly_global_price['global_price_usd_kg'] = (
    monthly_global_price['global_price_usd_lb'] / 0.453592
)

monthly_global_price = monthly_global_price[
    ['year', 'month', 'global_price_usd_kg']
]

monthly_global_price.head()


,year,month,global_price_usd_kg
0,2023,1,3.504190
1,2023,2,3.996257
2,2023,3,3.888009
3,2023,4,4.142765
4,2023,5,4.060391


Load Yield Data

In [6]:
# Historical Yield (2023–2024)
yield_hist = pd.read_csv(
    '/content/drive/MyDrive/Coffee/Dev/C1/yield/coffee_yield_simulation/yield_train_dataset.csv'
)

yield_hist = (
    yield_hist
    .groupby(['year', 'month'])
    .agg(predicted_yield_kg=('adjusted_yield_kg', 'sum'))
    .reset_index()
)

yield_hist['yield_source'] = 'historical'

# Future Yield (2025–2026)
yield_future = pd.read_csv(
    '/content/drive/MyDrive/Coffee/Dev/C1/yield/coffee_yield_simulation/yield_predictions_2025_2026.csv'
)

yield_future = (
    yield_future
    .groupby(['year', 'month'])
    .agg(predicted_yield_kg=('predicted_yield_kg', 'sum'))
    .reset_index()
)

yield_future['yield_source'] = 'predicted'

# Combine Yield Data
monthly_yield = pd.concat(
    [yield_hist, yield_future],
    ignore_index=True
).sort_values(['year', 'month'])

monthly_yield.head()


,year,month,predicted_yield_kg,yield_source
0,2023.0,1.0,573.95,historical
1,2023.0,2.0,667.74,historical
2,2023.0,3.0,684.29,historical
3,2023.0,4.0,590.14,historical
4,2023.0,5.0,720.92,historical


Merge All Core Inputs

In [7]:
price_data = (
    monthly_yield
    .merge(monthly_global_price, on=['year', 'month'], how='left')
    .merge(usd_lkr, on=['year', 'month'], how='left')
)


Handle Missing Exchange Rates (2026)


In [8]:
price_data['usd_lkr_rate'] = price_data['usd_lkr_rate'].ffill()


Generate Demand Index

In [9]:
def demand_index(month):
    if month in [11, 12, 1]:
        base = 1.1
    elif month in [6, 7]:
        base = 0.95
    else:
        base = 1.0

    noise = np.random.normal(0, 0.04)
    return round(np.clip(base + noise, 0.85, 1.15), 2)

price_data['demand_index'] = price_data['month'].apply(demand_index)


Generate Local Coffee Price

In [10]:
mean_yield = price_data['predicted_yield_kg'].mean()

def compute_local_price(row):
    if (
        pd.isna(row.global_price_usd_kg) or
        pd.isna(row.usd_lkr_rate) or
        pd.isna(row.predicted_yield_kg) or
        row.predicted_yield_kg <= 0
    ):
        return np.nan

    base_price = row.global_price_usd_kg * row.usd_lkr_rate

    supply_factor = mean_yield / row.predicted_yield_kg
    supply_factor = np.clip(supply_factor, 0.7, 1.3)

    return round(base_price * supply_factor * row.demand_index, 2)

price_data['local_coffee_price_lkr_per_kg'] = price_data.apply(
    compute_local_price, axis=1
)

Final Cleanup

In [11]:
price_data = price_data.replace([np.inf, -np.inf], np.nan)
price_data = price_data.dropna()

price_data.reset_index(drop=True, inplace=True)

Final Dataset

In [12]:
final_price_dataset = price_data[
    [
        'year',
        'month',
        'yield_source',
        'predicted_yield_kg',
        'global_price_usd_kg',
        'usd_lkr_rate',
        'demand_index',
        'local_coffee_price_lkr_per_kg'
    ]
]

final_price_dataset.head()

,year,month,yield_source,predicted_yield_kg,global_price_usd_kg,usd_lkr_rate,demand_index,local_coffee_price_lkr_per_kg
0,2023.0,1.0,historical,573.95,3.504190,365.0,1.12,1565.97
1,2023.0,2.0,historical,667.74,3.996257,362.0,0.99,1345.70
2,2023.0,3.0,historical,684.29,3.888009,324.0,1.03,1189.67
3,2023.0,4.0,historical,590.14,4.142765,320.0,1.06,1494.00
4,2023.0,5.0,historical,720.92,4.060391,290.0,0.99,1014.55


Save Dataset

In [13]:
output_dir = Path('/content/drive/MyDrive/Coffee/Dev/C2/Data/coffee_price_simulation')
output_dir.mkdir(exist_ok=True)

final_price_dataset.to_csv(
    output_dir / 'coffee_price_monthly_dataset_CORRECTED.csv',
    index=False
)

print("Corrected price dataset saved")


Corrected price dataset saved
